#### Upload data to DBFS in Databricks notebook

In [0]:
training = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/chuting3@illinois.edu/sample_linear_regression_data.txt')

In [0]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

In [0]:
lrModel = lr.fit(training)

In [0]:
lrModel.coefficients

Out[13]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [0]:
lrModel.intercept

Out[14]: 0.14228558260358093

In [0]:
training_summary = lrModel.summary

In [0]:
training_summary.r2

Out[16]: 0.027839179518600154

In [0]:
training_summary.rootMeanSquaredError

Out[17]: 10.16309157133015

#### Now split the data into training set and testing set

In [0]:
all_data = spark.read.format('libsvm').load('dbfs:/FileStore/shared_uploads/chuting3@illinois.edu/sample_linear_regression_data.txt')

In [0]:
train_data, test_data = all_data.randomSplit([0.7, 0.3])

In [0]:
train_data.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                 339|
|   mean|-0.10948873570543688|
| stddev|  10.187101445014868|
|    min| -28.571478869743427|
|    max|  27.111027963108548|
+-------+--------------------+



In [0]:
correct_model = lr.fit(train_data)

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
test_results.rootMeanSquaredError

Out[25]: 10.677827821457843

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [0]:
predictions = correct_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...| -2.2276060073767185|
|(10,[0,1,2,3,4,5,...|  -4.030856360949602|
|(10,[0,1,2,3,4,5,...| -0.7518955428672356|
|(10,[0,1,2,3,4,5,...| -0.9321829069251415|
|(10,[0,1,2,3,4,5,...|-0.08433538009313682|
|(10,[0,1,2,3,4,5,...|  -1.399553695752356|
|(10,[0,1,2,3,4,5,...|  -2.124752578523198|
|(10,[0,1,2,3,4,5,...|-0.18731715629523327|
|(10,[0,1,2,3,4,5,...|  0.9477417548521925|
|(10,[0,1,2,3,4,5,...|  1.9882836008794031|
|(10,[0,1,2,3,4,5,...|-0.48763098363781143|
|(10,[0,1,2,3,4,5,...|   3.715714052932664|
|(10,[0,1,2,3,4,5,...|  -3.138903818016554|
|(10,[0,1,2,3,4,5,...| -0.1303278604668744|
|(10,[0,1,2,3,4,5,...|  3.2842008182673896|
|(10,[0,1,2,3,4,5,...| -0.5636192469192682|
|(10,[0,1,2,3,4,5,...|  3.8952478045216763|
|(10,[0,1,2,3,4,5,...|  1.5156144924865156|
|(10,[0,1,2,3,4,5,...|  -4.214315164635412|
|(10,[0,1,2,3,4,5,...|  0.471827